In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import TargetEncoder
import numpy as np
import pandas as pd
# import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
# from shapely.geometry import Point
from sklearn.neighbors import BallTree
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [2]:
train_set = pd.read_csv('Datasets/train.csv')

train_set.head()

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,furnished,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region,monthly_rent
0,2021-09,jurong east,257,Jurong East Street 24,3 room,new generation,67.0,yes,1983,1.344518,103.738630,0.0,yuhua east,jurong east,west region,1600
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,yes,1978,1.330186,103.938717,0.0,bedok north,bedok,east region,2250
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,yes,1971,1.332242,103.845643,0.0,toa payoh central,toa payoh,central region,1900
3,2021-08,pasir ris,250,Pasir Ris Street 21,executive,apartment,149.0,yes,1993,1.370239,103.962894,0.0,pasir ris drive,pasir ris,east region,2850
4,2022-11,kallang/whampoa,34,Whampoa West,3-room,improved,68.0,yes,1972,1.320502,103.863341,0.0,bendemeer,kallang,central region,2100


In [3]:
# get copy

train_set_cleaned = train_set.copy()

In [4]:
train_set_cleaned[['rent_approval_year','rent_approval_month']] = train_set_cleaned['rent_approval_date'].str.split('-',expand=True)

In [5]:
# street name to lower case

train_set_cleaned['street_name'] = train_set_cleaned['street_name'].apply(str.lower)

In [6]:
# replace blank space with hyphen in flat_type (e.g. '2 room' to '2-room')

train_set_cleaned['flat_type'] = train_set_cleaned['flat_type'].str.replace(r'(2|3|4|5)-room|(\d) room', r'\1\2', regex=True)
train_set_cleaned['flat_type'] = train_set_cleaned['flat_type'].str.replace('executive', '6')
train_set_cleaned['flat_type'] = train_set_cleaned['flat_type'].astype(int)

In [7]:
train_set_cleaned = train_set_cleaned.drop(columns=['furnished', 'elevation', 'town', 'block', 'street_name', 'planning_area', 'subzone'])

In [8]:
train_set_cleaned.head(10)

,rent_approval_date,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,region,monthly_rent,rent_approval_year,rent_approval_month
0,2021-09,3,new generation,67.0,1983,1.344518,103.738630,west region,1600,2021,09
1,2022-05,4,new generation,92.0,1978,1.330186,103.938717,east region,2250,2022,05
2,2022-10,3,improved,67.0,1971,1.332242,103.845643,central region,1900,2022,10
3,2021-08,6,apartment,149.0,1993,1.370239,103.962894,east region,2850,2021,08
4,2022-11,3,improved,68.0,1972,1.320502,103.863341,central region,2100,2022,11
5,2023-04,6,premium apartment,130.0,2001,1.387847,103.764249,west region,2300,2023,04
6,2021-01,5,premium apartment,110.0,2005,1.388997,103.875148,north-east region,2100,2021,01
7,2022-06,3,new generation,67.0,1978,1.366048,103.838123,north-east region,2300,2022,06
8,2021-10,4,simplified,84.0,1987,1.344279,103.855556,central region,2100,2021,10
9,2021-04,5,premium apartment,112.0,2003,1.392832,103.910620,north-east region,2100,2021,04


In [9]:
region_ohe = OneHotEncoder(sparse=False)
fm_ohe = OneHotEncoder(sparse=False)
t_encoder = TargetEncoder()

def clean(X):
    X['rent_approval_date'] = X['rent_approval_date'].str[2:].str.replace('-', '', regex=False)
    X['rent_approval_date'] = X['rent_approval_date'].astype(int)
    X['street_name'] = X['street_name'].apply(str.lower)
    X['flat_type'] = X['flat_type'].str.replace(r'(2|3|4|5)-room|(\d) room', r'\1\2', regex=True)
    X['flat_type'] = X['flat_type'].str.replace('executive', '6')
    X['flat_type'] = X['flat_type'].astype(int)
    X = X.drop(columns=['furnished', 'elevation', 'town', 'block', 'street_name', 'planning_area', 'subzone'])
    
    return X
    
def one_hot_encode(X, isTest=False):
    if not isTest:
        region_ohe.fit(X[['region']])
         
    tr1 = region_ohe.transform(X[['region']])
    tr2 = pd.DataFrame(tr1, columns=region_ohe.get_feature_names_out(['region']))
    tr3 = pd.concat([X.reset_index(drop=True), tr2.reset_index(drop=True)], axis=1)
    tr3 = tr3.drop(columns=["region"])
    tr3.head()
    return tr3

def target_encode(X, isTest=False):
    if not isTest:
        t_encoder.fit(X['flat_model'], X['monthly_rent'])
        
    X['flat_model'] = t_encoder.transform(X['flat_model'])
    return X

def preprocess_data(X, isTest=False):
    X = clean(X)
    X = one_hot_encode(X, isTest)
    X = target_encode(X, isTest)
    return X


In [10]:
def prep_data_for_xgboost():
    training_data_raw = pd.read_csv('Datasets/train.csv')
    X_train, X_val = train_test_split(training_data_raw, test_size=0.2, random_state=42)
    
    X_train = preprocess_data(X_train)
    X_train, y_train = X_train.drop('monthly_rent', axis=1), X_train[['monthly_rent']]
    X_val = preprocess_data(X_val, True)
    X_val, y_val = X_val.drop('monthly_rent', axis=1), X_val[['monthly_rent']]
    
    print("Shape of training data: ", X_train.shape)
    print("Shape of training label: ", y_train.shape)
    print("Shape of validation data: ", X_val.shape)
    print("Shape of validation label: ", y_val.shape)

    return X_train, y_train, X_val, y_val

In [11]:
X_train, y_train, X_val, y_val = prep_data_for_xgboost()
X_train.head()

Shape of training data:  (48000, 12)
Shape of training label:  (48000, 1)
Shape of validation data:  (12000, 12)
Shape of validation label:  (12000, 1)


,rent_approval_date,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,region_central region,region_east region,region_north region,region_north-east region,region_west region
0,2107,3,2365.067275,67.0,1979,1.366600,103.855579,0.0,0.0,0.0,1.0,0.0
1,2201,5,2640.896729,110.0,2003,1.401158,103.751472,0.0,0.0,0.0,0.0,1.0
2,2107,3,2365.067275,67.0,1978,1.316994,103.768507,0.0,0.0,0.0,0.0,1.0
3,2103,3,2614.410973,74.0,1984,1.347162,103.742808,0.0,0.0,0.0,0.0,1.0
4,2206,4,2614.410973,93.0,2002,1.450165,103.819307,0.0,0.0,1.0,0.0,0.0


In [12]:
testing_data_raw = pd.read_csv('Datasets/test.csv')

In [13]:
X_test = preprocess_data(testing_data_raw, isTest=True)

In [14]:
xgb_model = xgb.XGBRegressor(
            n_estimators=1800, max_depth=5, eta=0.01, subsample=0.7, colsample_bytree=0.7,
            objective="reg:squarederror"
        )
xgb_model.fit(X_train, y_train)
y_val_pred = xgb_model.predict(X_val)
print('Validation loss:', np.sqrt(mean_squared_error(y_val, y_val_pred)))
y_train_pred = xgb_model.predict(X_train)
print('Training loss:', np.sqrt(mean_squared_error(y_train, y_train_pred)))

Validation loss: 483.1385787197614
Training loss: 465.3578866742624


#### Stacking models

In [15]:
# from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
# from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
# from sklearn.kernel_ridge import KernelRidge
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import RobustScaler, StandardScaler
# from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
# from sklearn.model_selection import KFold, cross_val_score

In [16]:
# # 定义一个交叉评估函数 Validation function
# n_folds = 5
# X = pd.concat((X_train, X_val),sort=False)
# y = pd.concat((y_train, y_val),sort=False)
# def rmsle_cv(model):
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
#     rmse= np.sqrt(-cross_val_score(model, X.values, y, scoring="neg_mean_squared_error", cv = kf))
#     return(rmse)

In [17]:
# # LASSO Regression
# lasso = make_pipeline(StandardScaler(), Lasso(alpha=0.1, random_state=42, max_iter=5000))
# score = rmsle_cv(lasso)
# print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [18]:
# # Gradient Boosting Regression
# GBoost = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# score = rmsle_cv(GBoost)
# print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std())) 

In [19]:
# # XGBoost     
# model_xgb = xgb.XGBRegressor(n_estimators=1800, max_depth=5, eta=0.01, subsample=0.7, colsample_bytree=0.7,
#             objective="reg:squarederror")

# score = rmsle_cv(model_xgb)
# print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [21]:
# # 堆叠模型(Stacking Averaged Models) 
# class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
#     def __init__(self, base_models, meta_model, n_folds=5):
#         self.base_models = base_models
#         self.meta_model = meta_model
#         self.n_folds = n_folds
        
   
#     def fit(self, X, y):
#         self.base_models_ = [list() for x in self.base_models]  #  4×5 list 存放训练好的模型
#         self.meta_model_ = clone(self.meta_model) # 复制基准模型，因为这里会有多个模型
#         kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
#         # 训练基准模型，基于基准模型训练的结果导出成特征
#         # that are needed to train the cloned meta-model
#         out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
#         for i, model in enumerate(self.base_models):
#             for train_index, holdout_index in kfold.split(X, y): #分为预测holdout_index和训练train_index
#                 instance = clone(model)
#                 self.base_models_[i].append(instance)
#                 instance.fit(X[train_index], y[train_index])
#                 y_pred = instance.predict(X[holdout_index])
#                 out_of_fold_predictions[holdout_index, i] = y_pred
                
#         # 将基准模型预测数据作为特征用来给meta_model训练
#         self.meta_model_.fit(out_of_fold_predictions, y)
#         return self
    
#     def predict(self, X):
#         meta_features = np.column_stack([
#             np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
#             for base_models in self.base_models_ ])
#         return self.meta_model_.predict(meta_features)

In [ ]:
# meta_model = model_xgb
# stacked_averaged_models = StackingAveragedModels(base_models = (GBoost, model_xgb),
#                                                  meta_model = meta_model,
#                                                 n_folds=10)
# score = rmsle_cv(stacked_averaged_models)
# print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
# final_pred = xgb_model.predict(X_test)  
# print("Length of final predictions is: ", len(final_pred))
# ids = np.arange(30000)
# df = pd.DataFrame({'Id': ids, 'Predicted': final_pred})
# df.to_csv("stack_team18_submission.csv", index=False)